<a href="https://colab.research.google.com/github/kplr-training/Airflow/blob/main/Ateliers/Solutions/04-Pipeline%20-%20Postgres.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Construction d'une pipeline d'exécution avec PostgreSQL 

![atelier4](https://user-images.githubusercontent.com/123757632/231912146-232774b4-c30c-4901-b4c1-ea27db59122b.png)

## Tâches de création de table

Il est possible d'utiliser PostgresOperator afin de définir des tâches qui créeront des tables dans la base de données postgres.

Il est prévu de créer deux tables : une pour faciliter le nettoyage des données (employees_temp) et une autre pour stocker les données nettoyées (employees).

In [ ]:
from airflow.providers.postgres.operators.postgres import PostgresOperator

create_employees_table = PostgresOperator(
    task_id="create_employees_table",
    postgres_conn_id="tutorial_pg_conn",
    sql="""
        CREATE TABLE IF NOT EXISTS employees (
            "Serial Number" NUMERIC PRIMARY KEY,
            "Company Name" TEXT,
            "Employee Markme" TEXT,
            "Description" TEXT,
            "Leave" INTEGER
        );""",
)

create_employees_temp_table = PostgresOperator(
    task_id="create_employees_temp_table",
    postgres_conn_id="tutorial_pg_conn",
    sql="""
        DROP TABLE IF EXISTS employees_temp;
        CREATE TABLE employees_temp (
            "Serial Number" NUMERIC PRIMARY KEY,
            "Company Name" TEXT,
            "Employee Markme" TEXT,
            "Description" TEXT,
            "Leave" INTEGER
        );""",
)

## Tâche de récupération de données

Ici, les données sont récupérées, enregistrées dans un fichier sur l'instance Airflow, puis chargées à partir de ce fichier dans une table intermédiaire afin de pouvoir exécuter les étapes de nettoyage des données.

In [ ]:
import os
import requests
from airflow.decorators import task
from airflow.providers.postgres.hooks.postgres import PostgresHook


@task
def get_data():
    # REMARQUE : configurez cela selon les besoins de votre environnement Airflow.
    data_path = "/opt/airflow/dags/files/employees.csv"
    os.makedirs(os.path.dirname(data_path), exist_ok=True)

    url = "https://raw.githubusercontent.com/apache/airflow/main/docs/apache-airflow/tutorial/pipeline_example.csv"

    response = requests.request("GET", url)

    with open(data_path, "w") as file:
        file.write(response.text)

    postgres_hook = PostgresHook(postgres_conn_id="tutorial_pg_conn")
    conn = postgres_hook.get_conn()
    cur = conn.cursor()
    with open(data_path, "r") as file:
        cur.copy_expert(
            "COPY employees_temp FROM STDIN WITH CSV HEADER DELIMITER AS ',' QUOTE '\"'",
            file,
        )
    conn.commit()

## Tâche de fusion de données
Ici, des enregistrements complètement uniques sont sélectionnés à partir des données récupérées, puis il est vérifié si des numéros de série d'employés sont déjà présents dans la base de données. Si tel est le cas, ces enregistrements sont mis à jour avec les nouvelles données.

In [ ]:
from airflow.decorators import task
from airflow.providers.postgres.hooks.postgres import PostgresHook


@task
def merge_data():
    query = """
        INSERT INTO employees
        SELECT *
        FROM (
            SELECT DISTINCT *
            FROM employees_temp
        ) t
        ON CONFLICT ("Serial Number") DO UPDATE
        SET
              "Employee Markme" = excluded."Employee Markme",
              "Description" = excluded."Description",
              "Leave" = excluded."Leave";
    """
    try:
        postgres_hook = PostgresHook(postgres_conn_id="tutorial_pg_conn")
        conn = postgres_hook.get_conn()
        cur = conn.cursor()
        cur.execute(query)
        conn.commit()
        return 0
    except Exception as e:
        return 1

## Finalisation du DAG

Les tâches ont été développées et doivent maintenant être intégrées dans un DAG. Ce DAG permet de définir quand et comment les tâches doivent s'exécuter, ainsi que les dépendances entre les tâches. Le DAG ci-dessous est configuré pour :

* s'exécuter tous les jours à minuit à partir du 24er mars 2023,

* ne s'exécuter qu'une seule fois en cas de jours manqués, et

* s'interrompre après 60 minutes.

À partir de la dernière ligne de la définition du DAG "process-employees", il est possible de voir :

In [ ]:
[create_employees_table, create_employees_temp_table] >> get_data() >> merge_data()

* la tâche "merge_data()" dépend de la tâche "get_data()",

* "get_data()" dépend à la fois des tâches "create_employees_table" et "create_employees_temp_table",

* et les tâches "create_employees_table" et "create_employees_temp_table" peuvent s'exécuter indépendamment.

En combinant tous ces éléments, le DAG est finalisé.

In [ ]:
import datetime
import pendulum
import os

import requests
from airflow.decorators import dag, task
from airflow.providers.postgres.hooks.postgres import PostgresHook
from airflow.providers.postgres.operators.postgres import PostgresOperator


@dag(
    dag_id="process-employees",
    schedule_interval="0 0 * * *",
    start_date=pendulum.datetime(2021, 1, 1, tz="UTC"),
    catchup=False,
    dagrun_timeout=datetime.timedelta(minutes=60),
)
def ProcessEmployees():
    create_employees_table = PostgresOperator(
        task_id="create_employees_table",
        postgres_conn_id="tutorial_pg_conn",
        sql="""
            CREATE TABLE IF NOT EXISTS employees (
                "Serial Number" NUMERIC PRIMARY KEY,
                "Company Name" TEXT,
                "Employee Markme" TEXT,
                "Description" TEXT,
                "Leave" INTEGER
            );""",
    )

    create_employees_temp_table = PostgresOperator(
        task_id="create_employees_temp_table",
        postgres_conn_id="tutorial_pg_conn",
        sql="""
            DROP TABLE IF EXISTS employees_temp;
            CREATE TABLE employees_temp (
                "Serial Number" NUMERIC PRIMARY KEY,
                "Company Name" TEXT,
                "Employee Markme" TEXT,
                "Description" TEXT,
                "Leave" INTEGER
            );""",
    )

    @task
    def get_data():
        # REMARQUE : configurez cela selon les besoins de votre environnement Airflow.
        data_path = "/opt/airflow/dags/files/employees.csv"
        os.makedirs(os.path.dirname(data_path), exist_ok=True)

        url = "https://raw.githubusercontent.com/apache/airflow/main/docs/apache-airflow/tutorial/pipeline_example.csv"

        response = requests.request("GET", url)

        with open(data_path, "w") as file:
            file.write(response.text)

        postgres_hook = PostgresHook(postgres_conn_id="tutorial_pg_conn")
        conn = postgres_hook.get_conn()
        cur = conn.cursor()
        with open(data_path, "r") as file:
            cur.copy_expert(
                "COPY employees_temp FROM STDIN WITH CSV HEADER DELIMITER AS ',' QUOTE '\"'",
                file,
            )
        conn.commit()

    @task
    def merge_data():
        query = """
            INSERT INTO employees
            SELECT *
            FROM (
                SELECT DISTINCT *
                FROM employees_temp
            ) t
            ON CONFLICT ("Serial Number") DO UPDATE
            SET "Serial Number" = excluded."Serial Number";
        """
        try:
            postgres_hook = PostgresHook(postgres_conn_id="tutorial_pg_conn")
            conn = postgres_hook.get_conn()
            cur = conn.cursor()
            cur.execute(query)
            conn.commit()
            return 0
        except Exception as e:
            return 1

    [create_employees_table, create_employees_temp_table] >> get_data() >> merge_data()


dag = ProcessEmployees()

Enregistrez ce code dans un fichier python dans le dossier /dags (par exemple dags/process-employees.py) et (après une courte attente), le DAG process-employees sera inclus dans la liste des DAG disponibles sur l'interface utilisateur web.




![image](https://user-images.githubusercontent.com/123757632/227974881-9e246848-cd68-4215-9dc6-e3083042455e.png)


Il est possible de déclencher le DAG "process-employees" en le sélectionnant (via le curseur à gauche) et en le lançant (en appuyant sur le bouton Exécuter sous Actions).


![image](https://user-images.githubusercontent.com/123757632/227974915-f7f4f7f6-6401-4ed1-a435-5c6db720d48c.png)


En examinant la vue en grille du DAG "process-employees", il est possible de constater que toutes les tâches ont été exécutées avec succès lors de toutes les exécutions effectuées. C'est un succès !


![image](https://user-images.githubusercontent.com/123757632/227974958-41e5b7bb-c5a0-4d22-acd3-8ae67b561586.png)